In [1]:
import tensorflow as tf
import numpy as np
import time

In [2]:
phn_61 = ['aa', 'ae', 'ah', 'ao', 'aw', 'ax', 'ax-h', 'axr', 'ay', 'b', 'bcl', 'ch', 'd', 'dcl', 'dh', 'dx', 'eh', 'el', 'em', 'en', 'eng', 'epi', 'er', 'ey', 'f', 'g', 'gcl', 'h#', 'hh', 'hv', 'ih', 'ix', 'iy', 'jh', 'k', 'kcl', 'l', 'm', 'n', 'ng', 'nx', 'ow', 'oy', 'p', 'pau', 'pcl', 'q', 'r', 's', 'sh', 't', 'tcl', 'th', 'uh', 'uw', 'ux', 'v', 'w', 'y', 'z', 'zh']
phn_39 = ['ae', 'ao', 'aw', 'ax', 'ay', 'b', 'ch', 'd', 'dh', 'dx', 'eh', 'er', 'ey', 'f', 'g', 'h#', 'hh', 'ix', 'iy', 'jh', 'k', 'l', 'm', 'n', 'ng', 'ow', 'oy', 'p', 'r', 's', 't', 'th', 'uh', 'uw', 'v', 'w', 'y', 'z', 'zh']
mapping = {'ah': 'ax', 'ax-h': 'ax', 'ux': 'uw', 'aa': 'ao', 'ih': 'ix', 'axr': 'er', 'el': 'l', 'em': 'm', 'en': 'n', 'nx': 'n', 'eng': 'ng', 'sh': 'zh', 'hv': 'hh', 'bcl': 'h#', 'pcl': 'h#', 'dcl': 'h#', 'tcl': 'h#', 'gcl': 'h#', 'kcl': 'h#', 'q': 'h#', 'epi': 'h#', 'pau': 'h#'}

TRAIN_FILE = './data/fbank/train.tfrecords'
DEV_FILE = './data/fbank/dev.tfrecords'
TEST_FILE = './data/fbank/test.tfrecords'
checkpoints_path = './model/ctc_residual/ckpt'
ft_checkpoints_path = './model/ctc_residual/finetunning/ckpt'

feat_type = 'fbank'
feats_dim = 39 if feat_type=='mfcc' else 123 # log filter bank + energy term

batch_size = 32
num_hidden = 128
n_hidden_layer = 3
learning_rate = 0.001
num_classes = len(phn_61)+1 # num of phoneme + blank
epochs = 100

In [3]:
class Model(object):
    def __init__(self, batch_size, num_hidden, n_hidden_layer, feats_dim, num_classes, learning_rate=0.001, optimizer=None,
                 phn_61=None, phn_39=None, mapping=None, file_type=None, model_type=None):
        
        iterator = self._get_iterator(batch_size, feats_dim, file_type, model_type)
        self.iterator_initializer = iterator.initializer
        
        batched_data = iterator.get_next()
        features = batched_data[0]
        labels = batched_data[1]
        feats_seq_len = tf.to_int32(batched_data[2])
        labels_sparse = self._get_sparse_tensor(labels, -1)
        
        logits = self._compute_logits(features, feats_seq_len, num_hidden, n_hidden_layer, num_classes)
        self.saver = tf.train.Saver() # save graph until logits
        
        if model_type=='train':
            self.loss = self._compute_loss(labels_sparse, feats_seq_len, logits)
            self.update_step = self._get_update_step(self.loss, learning_rate, optimizer)
        elif model_type=='eval':
            self.loss = self._compute_loss(labels_sparse, feats_seq_len, logits)
        else: # 'infer'
            self.per = self._compute_per(labels_sparse, feats_seq_len, logits, phn_61, phn_39, mapping)
        
    def _get_sparse_tensor(self, dense, default):
        indices = tf.to_int64(tf.where(tf.not_equal(dense, default)))
        vals = tf.to_int32(tf.gather_nd(dense, indices))
        shape = tf.to_int64(tf.shape(dense))
        return tf.SparseTensor(indices, vals, shape)
                
    def _get_iterator(self, batch_size, feats_dim, file_type, model_type):
        dataset = tf.contrib.data.TFRecordDataset(file_type)
        context_features = {'feats_seq_len': tf.FixedLenFeature([], dtype=tf.int64),
                           'labels_seq_len': tf.FixedLenFeature([], dtype=tf.int64)}
        sequence_features = {'features': tf.FixedLenSequenceFeature([feats_dim], dtype=tf.float32),
                            'labels': tf.FixedLenSequenceFeature([], dtype=tf.int64)}
        dataset = dataset.map(lambda serialized_example: tf.parse_single_sequence_example(serialized_example,
                                                                        context_features=context_features,
                                                                        sequence_features=sequence_features))
        dataset = dataset.map(lambda context, sequence: (sequence['features'], sequence['labels'],
                                                        context['feats_seq_len'], context['labels_seq_len']))
        def batching_func(x):
            return x.padded_batch(batch_size,
                                 padded_shapes=(tf.TensorShape([None, feats_dim]),
                                               tf.TensorShape([None]),
                                               tf.TensorShape([]),
                                               tf.TensorShape([])),
                                 padding_values=(tf.cast(0, tf.float32),
                                                tf.cast(-1, tf.int64),
                                                tf.cast(0, tf.int64),
                                                tf.cast(0, tf.int64)))
        def key_func(features, labels, feats_seq_len, labels_seq_len):
            f0 = lambda: tf.constant(0, tf.int64)
            f1 = lambda: tf.constant(1, tf.int64)
            f2 = lambda: tf.constant(2, tf.int64)
            f3 = lambda: tf.constant(3, tf.int64)
            f4 = lambda: tf.constant(4, tf.int64)
            f5 = lambda: tf.constant(5, tf.int64)
            f6 = lambda: tf.constant(6, tf.int64)
            
            return tf.case([(tf.less_equal(feats_seq_len, 200), f0),
                   (tf.less_equal(feats_seq_len, 250), f1),
                   (tf.less_equal(feats_seq_len, 300), f2),
                   (tf.less_equal(feats_seq_len, 350), f3),
                   (tf.less_equal(feats_seq_len, 400), f4),
                   (tf.less_equal(feats_seq_len, 500), f5)], default=f6)
        def reduce_func(bucket_id, windowed_data):
            return batching_func(windowed_data)
        
        if model_type=='train':
            dataset = dataset.shuffle(10000)
            batched_dataset = dataset.group_by_window(key_func=key_func, reduce_func=reduce_func, window_size=batch_size)
            batched_dataset = batched_dataset.shuffle(10000)
        else:
            batched_dataset = batching_func(dataset)
            
        return batched_dataset.make_initializable_iterator()
    
    def _compute_logits(self, features, feats_seq_len, num_hidden, n_hidden_layer, num_classes):
        self.keep_prob = tf.placeholder(tf.float32)
        self.training = tf.placeholder(tf.bool)
        
        def residual_block(inp, out_channels, strides=(1,1)):
            inp_channels = inp.get_shape().as_list()[3]
            
            out = tf.layers.batch_normalization(inp, training=self.training) 
            out = tf.nn.relu(out)
            out = tf.layers.conv2d(out, filters=out_channels, kernel_size=(3,3), strides=strides, padding='same')
            out = tf.layers.dropout(out, rate=1-self.keep_prob, training=self.training)
            out = tf.layers.batch_normalization(out, training=self.training) 
            out = tf.nn.relu(out)
            out = tf.layers.conv2d(out, filters=out_channels, kernel_size=(3,3), strides=(1,1), padding='same')
            out = tf.layers.dropout(out, rate=1-self.keep_prob, training=self.training)
            
            if strides != (1,1): # down-sample
                inp = tf.layers.max_pooling2d(inp, pool_size=strides, strides=strides, padding='same')
            if inp_channels != out_channels:
                inp = tf.layers.conv2d(inp, filters=out_channels, kernel_size=(1,1), strides=(1,1), padding='same')

            return out + inp
        
        features = tf.stack(tf.split(features, num_or_size_splits=3, axis=-1), axis=0) # shape = [3, batch, max_time, feats_dim/3]
        features = tf.transpose(features, [1,3,2,0]) # shape = [batch, feats_dim/3, max_time, channels]
        
        conv = tf.layers.conv2d(features, filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation=tf.nn.relu)
        conv = tf.layers.max_pooling2d(conv, pool_size=(3,1), strides=(3,1), padding='same') # 41->14
    
        res1 = residual_block(conv, 64, strides=(1,1)) #
        res2 = residual_block(res1, 64, strides=(1,1)) #
        res3 = residual_block(res2, 64, strides=(1,1)) #
        res4 = residual_block(res3, 64, strides=(1,1)) #
        #pooled = tf.layers.max_pooling2d(res4, pool_size=(3,1), strides=(3,1), padding='same') # 3 -> 1
        
        flattend = tf.transpose(res4, [0,2,1,3]) # shape = [batch, width, height, channels] = [None, None, 14, 200]
        flattend = tf.reshape(flattend, [tf.shape(flattend)[0], tf.shape(flattend)[1], 14*64])
        
        fc1 = tf.layers.dense(flattend, 512)
        fc1 = tf.layers.batch_normalization(fc1, training=self.training) 
        fc1 = tf.nn.relu(fc1)
        fc1 = tf.layers.dropout(fc1, rate=1-self.keep_prob, training=self.training)
        
        fc2 = tf.layers.dense(fc1, 256)
        fc2 = tf.layers.batch_normalization(fc2, training=self.training) 
        fc2 = tf.nn.relu(fc2)
        fc2 = tf.layers.dropout(fc2, rate=1-self.keep_prob, training=self.training)
        
        fc3 = tf.layers.dense(fc2, 128)
        fc3 = tf.layers.batch_normalization(fc3, training=self.training) 
        fc3 = tf.nn.relu(fc3)
        fc3 = tf.layers.dropout(fc3, rate=1-self.keep_prob, training=self.training)
        
        return tf.layers.dense(fc3, num_classes)
    
    def _compute_loss(self, labels_sparse, feats_seq_len, logits):
        return tf.reduce_mean(tf.nn.ctc_loss(labels=labels_sparse, inputs=logits, sequence_length=feats_seq_len, time_major=False))
    
    def _get_update_step(self, loss, learning_rate, optimizer):    
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            params = tf.trainable_variables()
            gradients = tf.gradients(loss, params)
            clipped_gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
            if optimizer == 'adam':
                opt = tf.train.AdamOptimizer(learning_rate)
            elif optimizer == 'sgd':
                opt = tf.train.GradientDescentOptimizer(learning_rate)
            elif optimizer =='momentum':
                opt = tf.train.MomentumOptimizer(learning_rate, momentum=0.9, use_nesterov=True)
            update_step = opt.apply_gradients(zip(clipped_gradients, params))
        return update_step
        
    def _compute_per(self, labels_sparse, feats_seq_len, logits, phn_61, phn_39, mapping):
        phn_61_tensor = tf.constant(phn_61, dtype=tf.string)
        phn_39_tensor = tf.constant(phn_39, dtype=tf.string)
        mapping_table = tf.contrib.lookup.HashTable(tf.contrib.lookup.KeyValueTensorInitializer(list(mapping.keys()), list(mapping.values())), default_value='')
        self.mapping_table_init = mapping_table.init
        
        logits = tf.transpose(logits, [1,0,2])
        decoded, _ = tf.nn.ctc_greedy_decoder(logits, feats_seq_len)
        decoded = tf.to_int32(decoded[0])
        
        def map_to_reduced_phn(p):
            val = mapping_table.lookup(phn_61_tensor[p])
            f1 = lambda: tf.to_int32(tf.reduce_min(tf.where(tf.equal(val, phn_39_tensor))))
            f2 = lambda: tf.to_int32(tf.reduce_min(tf.where(tf.equal(phn_61_tensor[p], phn_39_tensor))))
            return tf.cond(tf.not_equal(val, ''), f1, f2)

        decoded_reduced = tf.SparseTensor(decoded.indices, tf.map_fn(map_to_reduced_phn, decoded.values), decoded.dense_shape)
        labels_reduced = tf.SparseTensor(labels_sparse.indices, tf.map_fn(map_to_reduced_phn, labels_sparse.values), labels_sparse.dense_shape)
        return tf.reduce_sum(tf.edit_distance(decoded_reduced, labels_reduced, normalize=False)) , tf.to_float(tf.size(labels_reduced.values))

In [4]:
train_graph = tf.Graph()
dev_graph = tf.Graph()
test_graph = tf.Graph()

with train_graph.as_default():
    train_model = Model(batch_size, num_hidden, n_hidden_layer, feats_dim, num_classes, learning_rate=learning_rate, 
                 optimizer='adam', file_type=TRAIN_FILE, model_type='train')
    initializer = tf.global_variables_initializer()

with dev_graph.as_default():
    dev_model = Model(batch_size, num_hidden, n_hidden_layer, feats_dim, num_classes, learning_rate=learning_rate, 
                 file_type=DEV_FILE, model_type='eval')
with test_graph.as_default():
    test_model = Model(batch_size, num_hidden, n_hidden_layer, feats_dim, num_classes, learning_rate=learning_rate, 
                 phn_61=phn_61, phn_39=phn_39, mapping=mapping, file_type=TEST_FILE, model_type='infer')
    
train_sess = tf.Session(graph=train_graph)
train_sess.run(initializer)
dev_sess = tf.Session(graph=dev_graph)
test_sess = tf.Session(graph=test_graph)
test_sess.run(test_model.mapping_table_init)

for epoch in range(epochs):
    train_sess.run(train_model.iterator_initializer)
    train_loss = []
    start = time.time()
    while True:
        try:
            _, cost = train_sess.run([train_model.update_step, train_model.loss], feed_dict={train_model.keep_prob: 0.7,
                                                                                             train_model.training: True})
            train_loss.append(cost)
        except tf.errors.OutOfRangeError:
            end = time.time()
            log = "Epoch {}/{}:, \ntrain_loss={:.3f}, time = {:.0f}s"
            print(log.format(epoch+1, epochs, np.mean(train_loss), end-start))
            checkpoint_path = train_model.saver.save(train_sess, checkpoints_path, global_step=epoch+1)
            
            dev_model.saver.restore(dev_sess, checkpoint_path)
            dev_sess.run(dev_model.iterator_initializer)
            dev_loss = []
            start = time.time()
            while True:
                try:
                    cost = dev_sess.run(dev_model.loss, feed_dict={dev_model.keep_prob: 1.0, dev_model.training: False})
                    dev_loss.append(cost)
                except tf.errors.OutOfRangeError:
                    end = time.time()
                    log = "\tdev_loss={:.3f}, time={:.0f}s"
                    print(log.format(np.mean(dev_loss), end-start))
                    break
            
            test_model.saver.restore(test_sess, checkpoint_path)
            test_sess.run(test_model.iterator_initializer)
            test_per = []
            test_seq_len = []
            start = time.time()
            while True:
                try:
                    _per, seq_len = test_sess.run(test_model.per, feed_dict={test_model.keep_prob: 1.0, test_model.training: False})
                    test_per.append(_per)
                    test_seq_len.append(seq_len)
                except tf.errors.OutOfRangeError:
                    end = time.time()
                    log = '\t\ttest_per={:.3f}, time={:.0f}s'
                    print(log.format(sum(test_per)/sum(test_seq_len), end-start))
                    break
            
            break
            
train_sess.close()
dev_sess.close()
test_sess.close()

Epoch 1/100:, 
train_loss=213.813, time = 41s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/ckpt-1
	dev_loss=274.056, time=2s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/ckpt-1
		test_per=0.870, time=5s
Epoch 2/100:, 
train_loss=113.049, time = 35s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/ckpt-2
	dev_loss=123.419, time=2s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/ckpt-2
		test_per=0.902, time=5s
Epoch 3/100:, 
train_loss=84.674, time = 36s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/ckpt-3
	dev_loss=101.224, time=2s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/ckpt-3
		test_per=0.787, time=5s
Epoch 4/100:, 
train_loss=73.262, time = 36s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/ckpt-4
	dev_loss=68.907, time=2s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/ckpt-4
		test_per=0.561, time=6s
Epoch 5/100:, 
train_loss=66.243, time = 36s
IN

	dev_loss=38.436, time=2s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/ckpt-35
		test_per=0.261, time=8s
Epoch 36/100:, 
train_loss=39.596, time = 35s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/ckpt-36
	dev_loss=38.126, time=2s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/ckpt-36
		test_per=0.260, time=8s
Epoch 37/100:, 
train_loss=39.498, time = 35s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/ckpt-37
	dev_loss=38.993, time=2s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/ckpt-37
		test_per=0.268, time=8s
Epoch 38/100:, 
train_loss=39.105, time = 35s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/ckpt-38
	dev_loss=37.796, time=2s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/ckpt-38
		test_per=0.255, time=8s
Epoch 39/100:, 
train_loss=38.944, time = 35s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/ckpt-39
	dev_loss=37.614, time=2s
INFO:tensorflo

KeyboardInterrupt: 

In [5]:
restored_ckpt_path = './model/ctc_residual/ckpt-49'
ft_learning_rate = 1e-3
ft_batch_size = 32
ft_epochs = 100

In [6]:
ft_train_graph = tf.Graph()
ft_dev_graph = tf.Graph()
ft_test_graph = tf.Graph()

with ft_train_graph.as_default():
    ft_train_model = Model(ft_batch_size, num_hidden, n_hidden_layer, feats_dim, num_classes, ft_learning_rate, optimizer='momentum',
                        file_type=TRAIN_FILE, model_type='train')
    initializer = tf.global_variables_initializer()
    
with ft_dev_graph.as_default():
    ft_dev_model = Model(ft_batch_size, num_hidden, n_hidden_layer, feats_dim, num_classes, file_type=DEV_FILE, model_type='eval')

with ft_test_graph.as_default():
    ft_test_model = Model(ft_batch_size, num_hidden, n_hidden_layer, feats_dim, num_classes, 
                       phn_61=phn_61, phn_39=phn_39, mapping=mapping, file_type=TEST_FILE, model_type='infer')

ft_train_sess = tf.Session(graph=ft_train_graph)
ft_train_sess.run(initializer)
ft_dev_sess = tf.Session(graph=ft_dev_graph)
ft_test_sess = tf.Session(graph=ft_test_graph)
ft_test_sess.run(ft_test_model.mapping_table_init)

ft_train_model.saver.restore(ft_train_sess, restored_ckpt_path)

for epoch in range(ft_epochs):
    ft_train_sess.run(ft_train_model.iterator_initializer)
    ft_train_loss  = []
    start = time.time()
    while True:
        try:
            _, cost = ft_train_sess.run([ft_train_model.update_step, ft_train_model.loss], feed_dict={ft_train_model.keep_prob: 0.7,
                                                                                                     ft_train_model.training: True})
            ft_train_loss.append(cost)
        except tf.errors.OutOfRangeError:
            end = time.time()
            log = "Epoch {}/{}:, \ntrain_loss={:.3f}, time = {:.0f}s"
            print(log.format(epoch+1, ft_epochs, np.mean(ft_train_loss), end-start))
            ft_checkpoint_path = ft_train_model.saver.save(ft_train_sess, ft_checkpoints_path, global_step=epoch+1)
            
            ft_dev_model.saver.restore(ft_dev_sess, ft_checkpoint_path)
            ft_dev_sess.run(ft_dev_model.iterator_initializer)
            ft_dev_loss = []
            start = time.time()
            while True:
                try:
                    cost = ft_dev_sess.run(ft_dev_model.loss, feed_dict={ft_dev_model.keep_prob: 1.0, ft_dev_model.training: False})
                    ft_dev_loss.append(cost)
                except tf.errors.OutOfRangeError:
                    end = time.time()
                    log = "\tdev_loss={:.3f}, time = {:.0f}s"
                    print(log.format(np.mean(ft_dev_loss), end-start))
                    break
                
            ft_test_model.saver.restore(ft_test_sess, ft_checkpoint_path)
            ft_test_sess.run(ft_test_model.iterator_initializer)
            ft_test_per = []
            ft_test_seq_len = []
            start = time.time()
            while True:
                try:
                    _per, seq_len = ft_test_sess.run(ft_test_model.per, feed_dict={ft_test_model.keep_prob: 1.0, ft_test_model.training: False})
                    ft_test_per.append(_per)
                    ft_test_seq_len.append(seq_len)
                except tf.errors.OutOfRangeError:
                    end = time.time()
                    log = '\t\ttest_per={:.3f}, time={:.0f}s'
                    print(log.format(sum(ft_test_per)/sum(ft_test_seq_len), end-start))
                    break
            # go to netxt epoch
            break
        
        
ft_train_sess.close()
ft_dev_sess.close()
ft_test_sess.close()

INFO:tensorflow:Restoring parameters from ./model/ctc_residual/ckpt-49
Epoch 1/100:, 
train_loss=35.855, time = 35s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/finetunning/ckpt-1
	dev_loss=35.725, time = 2s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/finetunning/ckpt-1
		test_per=0.245, time=8s
Epoch 2/100:, 
train_loss=35.341, time = 35s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/finetunning/ckpt-2
	dev_loss=35.508, time = 2s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/finetunning/ckpt-2
		test_per=0.243, time=8s
Epoch 3/100:, 
train_loss=35.220, time = 35s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/finetunning/ckpt-3
	dev_loss=35.502, time = 2s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/finetunning/ckpt-3
		test_per=0.241, time=8s
Epoch 4/100:, 
train_loss=35.101, time = 35s
INFO:tensorflow:Restoring parameters from ./model/ctc_residual/finetunning/ckpt-4
	dev_loss=35.

KeyboardInterrupt: 